In [1]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs.git

Cloning into 'german-traffic-signs'...
Unpacking objects: 100% (6/6), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import pickle
import random
import cv2

In [ ]:
np.random.seed(2222)

In [ ]:
!ls german-traffic-signs

In [ ]:
with open('german-traffic-signs/train.p', 'rb') as f:
    train_data=pickle.load(f)

with open('german-traffic-signs/valid.p', 'rb') as f:
    val_data=pickle.load(f)

with open('german-traffic-signs/test.p', 'rb') as f:
    test_data=pickle.load(f)

X_train, y_train = train_data['features'], train_data['labels']
X_val, y_val = val_data['features'], val_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
all = X_train.shape[0]+X_val.shape[0]+X_test.shape[0]
print('X_train percent = ',round(X_train.shape[0]/all*100, 2))
print('X_val percent = ',round(X_val.shape[0]/all*100, 2))
print('X_test percent = ',round(X_test.shape[0]/all*100, 2))

In [ ]:
assert(X_train.shape[0]==y_train.shape[0]), 'The number of images is not equal to the number of labels'
assert(X_val.shape[0]==y_val.shape[0]), 'The number of images is not equal to the number of labels'
assert(X_test.shape[0]==y_test.shape[0]), 'The number of images is not equal to the number of labels'
assert(X_train.shape[1:]==(32, 32, 3)), 'The dimensions of the images are not 32 x 32 x 3'
assert(X_test.shape[1:]==(32, 32, 3)), 'The dimensions of the images are not 32 x 32 x 3'
assert(X_val.shape[1:]==(32, 32, 3)), 'The dimensions of the images are not 32 x 32 x 3'

In [ ]:
data=pd.read_csv('german-traffic-signs/signnames.csv')

In [ ]:
data

In [ ]:
num_of_samples = []

cols = 5
num_classes = 43

fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(10, 50))
fig.tight_layout()
for i in range(cols):
    for j, row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + '-'+row['SignName'])
            num_of_samples.append(len(x_selected))

In [ ]:
print(num_of_samples)
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")
plt.show()

In [ ]:
np.std(num_of_samples)

In [ ]:
print(np.max(num_of_samples), np.min(num_of_samples))

## preprocessing

In [ ]:
plt.imshow(X_train[222])
plt.axis('off')
print(X_train[222].shape)
print(y_train[222])

### convert to gray

In [ ]:
def grayscale(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img

img = grayscale(X_train[222])
plt.imshow(img, cmap=plt.get_cmap('gray'))
plt.axis('off')
print(img.shape)

### histogram equalization

In [ ]:
def equalize(img):
    # equalizeHist funciont only accept gray images
    equalized_img = cv2.equalizeHist(img)
    return equalized_img

img = equalize(img)
print(img.shape)
plt.imshow(img, cmap=plt.get_cmap('gray'))
plt.axis('off')
plt.show()

In [ ]:
def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

X_train=np.array(list(map(preprocessing, X_train)))
X_val=np.array(list(map(preprocessing, X_val)))
X_test=np.array(list(map(preprocessing, X_test)))

In [ ]:
plt.imshow(X_train[random.randint(0, len(X_train)-1)], cmap=plt.get_cmap('gray'))
plt.axis('off')
print(X_train.shape)
plt.show()

In [ ]:
X_train=X_train.reshape(34799, 32, 32, 1)
X_test=X_test.reshape(X_test.shape[0], 32, 32, 1)
X_val=X_val.reshape(X_val.shape[0], 32, 32, 1)

In [ ]:
data_gen=ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10)
data_gen.fit(X_train)

In [ ]:
batches = data_gen.flow(X_train, y_train, batch_size=20)
X_batch, y_batch = next(batches)
fig, ax = plt.subplots(1, 15, figsize=(20, 5))
fig.tight_layout()
for i in range(15):
    ax[i].imshow(X_batch[i].reshape(32, 32), cmap=plt.get_cmap('gray'))
    ax[i].axis('off')

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
y_train=to_categorical(y_train, 43)
y_test=to_categorical(y_test, 43)
y_val=to_categorical(y_val, 43)

In [ ]:
def modified_model():
    model = Sequential()
    model.add(Conv2D(60, (5, 5), input_shape=(32, 32, 1), activation='relu'))
    model.add(Conv2D(60, (5, 5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    # model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = modified_model()
print(model.summary())

In [ ]:
history=model.fit_generator(data_gen.flow(X_train, y_train, batch_size=50), steps_per_epoch=len(X_train)//50, epochs=13, validation_data=(X_val, y_val), shuffle=True)

In [ ]:
score=model.evaluate(X_test, y_test, verbose=0)
print('accuracy = ', round(score[1]*100, 3),'%', '\nloss = ', round(score[0], 3))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['traning', 'validation'])
plt.title('loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['traning', 'validation'])
plt.title('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.save('model2')

In [ ]:
#fetch image

import requests
from PIL import Image
url = 'https://c8.alamy.com/comp/G667W0/road-sign-speed-limit-30-kmh-zone-passau-bavaria-germany-G667W0.jpg'
url2 = 'https://c8.alamy.com/comp/A0RX23/cars-and-automobiles-must-turn-left-ahead-sign-A0RX23.jpg'
url3 = 'https://previews.123rf.com/images/bwylezich/bwylezich1608/bwylezich160800375/64914157-german-road-sign-slippery-road.jpg'
url4 = 'https://previews.123rf.com/images/pejo/pejo0907/pejo090700003/5155701-german-traffic-sign-no-205-give-way.jpg'
url5 = 'https://c8.alamy.com/comp/J2MRAJ/german-road-sign-bicycles-crossing-J2MRAJ.jpg'
r = requests.get(url4, stream=True)
img = Image.open(r.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
#Preprocess image

img = np.asarray(img)
img = cv2.resize(img, (32, 32))
img = preprocessing(img)
plt.imshow(img, cmap = plt.get_cmap('gray'))
print(img.shape)

In [ ]:
#Reshape reshape

img = img.reshape(1, 32, 32, 1)

In [ ]:
#Test image
print("predicted sign: "+ str(np.argmax(model.predict(img))))

In [ ]:
data.loc[23]['SignName']

In [ ]:
model2 = keras.models.load_model('model2')

In [ ]:
! zip -r model2.zip model2